# Exercise 3

In [1]:
import pandas as pd
import numpy as np
import random
from dateutil.relativedelta import relativedelta
from datetime import date, timedelta, datetime
import plotly.graph_objs as go
from faker import Faker
import time
import mysql.connector
from IPython.display import display, clear_output
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

In [2]:
# Connect to the MySQL database
# Customer_details : id, name, phonenumber, date of birth, age
# Location_details : id, latitude, Longitude, Location, Time_stamp
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="1234",
    database="spa"
)

# Create a cursor to interact with the database
cursor = db.cursor()

In [3]:
# Calculate Age using Date of Birth
def calculate_age(birth_date):
    today = date.today()
    age = relativedelta(today, birth_date)
    return age.years

In [4]:
# Generate random customer locations with phone numbers and location names
def generate_customer_location(customer_id):
    location_name, (latitude, longitude) = random.choice(list(locations.items()))
    return customer_id, latitude, longitude, location_name

In [5]:
# All locations in the Mall
locations = {
    "Ice Cream Shop": (37.7749, -122.4194),
    "Book Store": (37.7732, -122.4186),
    "Pizzario": (37.7750, -122.4195),
    "Clothing Store": (37.7735, -122.4200),
    "Electronics Store": (37.7740, -122.4170),
}

# Number of Customers and Simulation Duration
num_customers = 50
simulation_duration = 60

In [6]:
# Generating customer details
number = {}
fake = Faker()

for id in range(1, num_customers + 1):
    phone_number = f"+91-{random.randint(100, 999)}-{random.randint(100, 999)}-{random.randint(1000, 9999)}"
    
    dob = date.today() - timedelta(days=random.randint(365 * 18, 365 * 71))
    age = calculate_age(dob)
    
    name = fake.name()
    
    details_query = "INSERT INTO customer_details(customerid, customername, phonenumber, dob, age) VALUES (%s, %s, %s, %s, %s)"
    insert_data = (id, name, phone_number, dob, age)
    
    cursor.execute(details_query, insert_data)
    db.commit()    

# Simulation

In [18]:
def collect_data():
    start_time = time.time()
    timestamp_threshold = datetime.now()
    location_ = ['Book Store','Clothing Store','Electronics Store','Ice Cream Shop', 'Pizza Store']
    df = pd.DataFrame(columns=['customer_id', 'latitude', 'longitude', 'location_name', 'timestamp'])

    # Initialize the bar chart
    fig_line = go.FigureWidget()
    fig_line.update_layout(
        title="Number of People in Each Location",
        xaxis=dict(title="Location"),
        yaxis=dict(title="Number of People"),
    )

    display(fig_line)
    legends = {}
    # print(timestamp_threshold)
    while time.time() - start_time < simulation_duration:

        # Generate random locations with latitude, longitude, phone numbers, and location names for each customer
        for customer_id in range(1, num_customers + 1):
            customer_id, latitude, longitude, location_name = generate_customer_location(customer_id)

            date_time = datetime.now()

            # Insert customer data into the database
            insert_query = "INSERT INTO location_details (customerid, latitude, longitude, location, timestamp_) VALUES (%s, %s, %s, %s, %s)"
            location_data = (customer_id, latitude, longitude, location_name, date_time)
            cursor.execute(insert_query, location_data)
            db.commit()

        query = "SELECT * FROM location_details WHERE timestamp_ > %s"
        timestamp_threshold = df['timestamp'].max() if not df.empty else timestamp_threshold
        cursor.execute(query, (timestamp_threshold,))

        new_data = pd.DataFrame(cursor.fetchall(), columns=['customer_id', 'latitude', 'longitude', 'location_name', 'timestamp'])

        # Append new data to the DataFrame
        if not new_data.empty:
            df = pd.concat([df, new_data], ignore_index=True)

            # Calculate and update the bar chart with the number of people in each location
            location_counts = new_data['location_name'].value_counts()
            location_values = {}

            for val in location_:
                if val not in location_counts.index:
                    location_values[val] = 0
                else:
                    location_values[val] = location_counts[val]

            with fig_line.batch_update():
                for location in location_:
                    location_counts = df[df['location_name'] == location]['timestamp'].value_counts().sort_index()
                    if location not in legends:
                        legends[location] = True
                    else:
                        legends[location] = False
                    fig_line.add_trace(go.Scatter(x=location_counts.index, y=location_counts.values, mode='lines+markers', name=location, showlegend=legends[location]))
                fig_line.update_layout(title=f"Number of People in Each Location (Line Chart) ({new_data['timestamp'].max()})")

        time.sleep(1)

In [13]:
# Cleaning
def data_cleaning():
    
    cursor.execute("select * from customer_details")
    customer_data = cursor.fetchall()
    customer_data = pd.DataFrame(customer_data, columns = ["Customer Id", "Name", "Phone Number", "Date of Birth", "Age"])
    
    cursor.execute("select * from location_details")
    location_data = cursor.fetchall()
    location_data = pd.DataFrame(location_data, columns = ["Customer Id", "Latitude", "Longitude", "Location", "TimeStamp"])
    
    # Filling null values
    customer_data.bfill(axis = 'rows')
    location_data.bfill(axis = 'rows')
    
    location_data = location_data[location_data["Latitude"] >= 25]
    location_data = location_data[location_data["Longitude"] <= 10]
    
    locations = ['Book Store','Clothing Store','Electronics Store','Ice Cream Shop', 'Pizza Store']
    location_data = location_data[location_data["Location"].isin(locations)]
    
    return customer_data, location_data

In [14]:
# Transformation
def data_transformation(customer_data, location_data):
    scaler = MinMaxScaler()
    label = LabelEncoder()
    
    columns = location_data.columns
    
    location_data["Year"] = pd.DatetimeIndex(location_data["TimeStamp"]).year
    location_data["Month"] = pd.DatetimeIndex(location_data["TimeStamp"]).month
    location_data["Day"] = pd.DatetimeIndex(location_data["TimeStamp"]).day
    location_data["Hour"] = pd.DatetimeIndex(location_data["TimeStamp"]).hour
    
    scale_columns = ['Latitude', 'Longitude', 'Year', 'Month', 'Day', 'Hour']
    
    locations = pd.get_dummies(location_data["Location"], drop_first=True)
    location_data[scale_columns] = scaler.fit_transform(location_data[scale_columns])
    location_data.drop(["TimeStamp"], inplace = True, axis = 1)    
    
    return customer_data, location_data

In [15]:
# Processing
def data_processing():
    # Average Age by Location using window functions
    query1 = """ select distinct(ld.location), avg(cd.age) over (partition BY ld.location) AS avg_age
                 from customer_details cd
                 inner join location_details ld ON cd.customerid = ld.customerid;
            """

    avg_age_by_location = pd.read_sql(query1, db)
    
    # Customer in Electronics store or Book Store
    query2 = """ select cd.phonenumber, cd.customerid, cd.customername, cd.age 
                 from customer_details cd
                 inner join location_details ld on cd.customerid = ld.customerid
                 where (ld.location = "Electronics Store" or ld.location = "Book Store")"""
    
    data_in_location = pd.read_sql(query2, db)

    #grouping w.r.t location and finding number of customer between the time of 1pm to 3pm
    query3 = """select ld.location, count(distinct(cd.customerid)) as count
                from customer_details cd 
                inner join location_details ld on cd.customerid = ld.customerid
                where ld.timestamp_ >= "2023-09-10 15:7:00" and ld.timestamp_ <= "2023-09-10 15:30:00"
                group by ld.location
             """
    group_location = pd.read_sql(query3, db)
    
    
    # Customer Age below 26
    query4 = """select cd.phonenumber, cd.customername 
                from customer_details cd
                inner join location_details ld on cd.customerid = ld.customerid
                where cd.age <= 26"""
    data_under_26 = pd.read_sql(query4, db)
    

In [16]:
# Pipeline
def streaming_pipeline():
    collect_data()
    customer_data, location_data = data_cleaning()
    customer_data, location_data = data_transformation(customer_data, location_data)
    data_processing()

In [19]:
streaming_pipeline()

FigureWidget({
    'data': [],
    'layout': {'template': '...',
               'title': {'text': 'Number of People in Each Location'},
               'xaxis': {'title': {'text': 'Location'}},
               'yaxis': {'title': {'text': 'Number of People'}}}
})

In [81]:
# cursor.close()
db.close()